In [ ]:
using HDF5
using PyPlot

In [ ]:
function combine_hdf5_files(file_paths::Vector{String}, output_path::String)
    # Use sets to collect unique values
    wavelength_set = Set{Float64}()
    ice_shell_thickness_set = Set{Float64}()
    
    combined_data = Dict(
        "Wavelength" => Float64[],
        "Ice Shell Thickness" => Float64[],
        "Viscous Relaxation Time(Half-Space)" => Float64[],
        "Viscous Relaxation Time(Model)" => Float64[],
        "Fitted Viscous Relaxation Time" => Float64[],
        "Thickening Time" => Float64[]
    )

    for file_path in file_paths
        h5open(file_path, "r") do file
            g = file["Model Run"]
            append!(combined_data["Wavelength"], read(g["Wavelength"]))
            append!(combined_data["Ice Shell Thickness"], read(g["Ice Shell Thickness"]))
            append!(combined_data["Viscous Relaxation Time(Half-Space)"], read(g["Viscous Relaxation Time(Half-Space)"]))
            append!(combined_data["Viscous Relaxation Time(Model)"], read(g["Viscous Relaxation Time(Model)"]))
            append!(combined_data["Fitted Viscous Relaxation Time"], read(g["Fitted Viscous Relaxation Time"]))
            append!(combined_data["Thickening Time"], read(g["Thickening Time"]))  
            # Add to sets to ensure uniqueness
            union!(wavelength_set, read(g["Wavelength"]))
            union!(ice_shell_thickness_set, read(g["Ice Shell Thickness"]))
        end
    end

    # Convert sets to sorted arrays
    sorted_wavelength = sort(collect(wavelength_set))
    sorted_hice = sort(collect(ice_shell_thickness_set))

    # Initialize matrices for the results
    n_wavelength = length(sorted_wavelength)
    n_hice = length(sorted_hice)
    
    t_hs_matrix = zeros(n_wavelength,n_hice)
    t_rel_matrix = zeros(n_wavelength,n_hice)
    t_fit_matrix = zeros(n_wavelength,n_hice)
    t_thic_matrix = zeros(n_wavelength,n_hice)

    # Fill the matrices
    for i in 1:length(combined_data["Viscous Relaxation Time(Half-Space)"])
        wavelength = combined_data["Wavelength"][i]
        hice = combined_data["Ice Shell Thickness"][i]   
        # Find the correct indices for wavelength and hice
        j_idx = findfirst(x -> x == hice, sorted_hice)
        i_idx = findfirst(x -> x == wavelength, sorted_wavelength)
        # Fill the matrices
        t_hs_matrix[i_idx, j_idx] = combined_data["Viscous Relaxation Time(Half-Space)"][i]
        t_rel_matrix[i_idx, j_idx] = combined_data["Viscous Relaxation Time(Model)"][i]
        t_fit_matrix[i_idx, j_idx] = combined_data["Fitted Viscous Relaxation Time"][i]
        t_thic_matrix[i_idx, j_idx] = combined_data["Thickening Time"][i]
    end

    # Write the sorted unique data into a new HDF5 file
    h5open(output_path, "w") do file
        g = create_group(file, "Combined Model Run")
        g["Wavelength"] = sorted_wavelength
        g["Ice Shell Thickness"] = sorted_hice
        g["Viscous Relaxation Time(Half-Space)"] = t_hs_matrix
        g["Viscous Relaxation Time(Model)"] = t_rel_matrix
        g["Fitted Viscous Relaxation Time"] = t_fit_matrix
        g["Thickening Time"] = t_thic_matrix
        attrs(g)["Description"] = "This group contains combined and sorted unique datasets"
    end
end


In [ ]:
file_paths = ["/Users/christianaguirre/ice_model_code/Model_Outputs/h_5.0_lambda_5.0_amp_20.0/data.hdf5",
                "/Users/christianaguirre/ice_model_code/Model_Outputs/h_5.0_lambda_20.0_amp_20.0/data.hdf5",
                "/Users/christianaguirre/ice_model_code/Model_Outputs/h_20.0_lambda_5.0_amp_20.0/data.hdf5",
                "/Users/christianaguirre/ice_model_code/Model_Outputs/h_20.0_lambda_20.0_amp_20.0/data.hdf5"]
output_path = "combined_data.hdf5"
combine_hdf5_files(file_paths,output_path)

In [ ]:
fname = "/Users/christianaguirre/ice_model_code/combined_data.hdf5"
# Displaying all of the file's information
fid = h5open(fname, "r")

In [ ]:
# Reading Data from file
modeldata = fid["Combined Model Run"];

# Displaying the contents in Model Run group 
for obj in modeldata
    data = read(obj)
    println(obj)
    display(data)
end

# # Accessing Model Run contents that I want
Wavelength = read(modeldata,"Wavelength");
Ice_shell = read(modeldata, "Ice Shell Thickness");
Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening = read(modeldata, "Thickening Time");
Time_rel_fitted = read(modeldata, "Fitted Viscous Relaxation Time");

# Close file
close(fid)

In [ ]:
println("Wavelength: ",Wavelength/1e3,"(km)")
println("Ice Shell Thickness: ",Ice_shell/1e3,"(km)")
println("Viscous Relaxation Time (Half-Space): ",Time_halfspace/1e3,"(kyr)")
println("Viscous Relaxation Time (Model): ",Time_rel/1e3,"(kyr)")
println("Thickening Time: ",Time_thickening/1e3,"(kyr)")
println("Fitted Viscous Relaxation Time: ",Time_rel_fitted/1e3,"(kyr)")

In [ ]:
# is in time yrs 
# time/1e6 is Millions of Years
# time/1e3 is Thousands of Years

In [ ]:
figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_rel./Time_thickening)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t'")
colorbar(label=L"t'",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_rel/1e3)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{relaxation}")
colorbar(label=L"t_{relaxation}(kyr)",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()

figure()
cs = contour(Ice_shell/1e3,Wavelength/1e3,Time_thickening/1e3)
clabel(cs,inline=true,fmt="%0.1f                       ",colors="w",fontsize=10)
contourf(cs,cmap=get_cmap("viridis"))
title(L"t_{thickening}")
colorbar(label=L"t_{thickening}(kyr)",location="right")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda\,(km)")
gca().set_xlabel(L"h_{ice\,shell}\,(km)",rotation=0)
# gca().yaxis.set_label_coords(-0.23,0.5)
gca().spines["top"].set_visible(false) # Hide the top edge of the axis
gca().spines["right"].set_visible(false) # Hide the right edge of the axis
gca().xaxis.set_ticks_position("bottom")
gca().yaxis.set_ticks_position("left")
gca().spines["left"].set_position(("axes",-0.03)) # Offset the left scale from the axis
gca().spines["bottom"].set_position(("axes",-0.03)) # Offset the bottom scale from the axis
# gca().spines["top"].set_color("none") # Remove the top axis boundary
# gca().spines["right"].set_color("none") # Remove the right axis boundary
# gca().set_xlim([10.0,30.0])
# gca().set_ylim([10.0,100.0])
tight_layout()
# savefig("Trel_Ttic.jpeg",dpi=300)
show()